# Exercise 2: Confidence Intervals - NHANES


This exercise, we are going to practice on how to load data, clean/manipulate a dataset, and construct a confidence interval for the difference between two population proportions and means.

We will use the 2015-2016 wave of the NHANES data for our analysis.

For our population proportions, we will analyze the difference of proportion between female and male smokers.  The column that specifies smoker and non-smoker is "SMQ020" in our dataset.

For our population means, we will analyze the difference of mean of body mass index within our female and male populations.  The column that includes the body mass index value is "BMXBMI".

Additionally, the gender is specified in the column "RIAGENDR".

In [1]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [2]:
url = "nhanes_2015_2016.csv"
da = pd.read_csv(url)

In [3]:
da.head()

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
0,83732,1.0,NaN,1.0,1,1,62,3,1.0,5.0,...,124.0,64.0,94.8,184.5,27.8,43.3,43.6,35.9,101.1,2.0
1,83733,1.0,NaN,6.0,1,1,53,3,2.0,3.0,...,140.0,88.0,90.4,171.4,30.8,38.0,40.0,33.2,107.9,NaN
2,83734,1.0,NaN,NaN,1,1,78,3,1.0,3.0,...,132.0,44.0,83.4,170.1,28.8,35.6,37.0,31.0,116.5,2.0
3,83735,2.0,1.0,1.0,2,2,56,3,1.0,5.0,...,134.0,68.0,109.8,160.9,42.4,38.5,37.7,38.3,110.1,2.0
4,83736,2.0,1.0,1.0,2,2,42,4,1.0,4.0,...,114.0,54.0,55.2,164.9,20.3,37.4,36.0,27.2,80.4,2.0


In [15]:
da.columns

Index(['SEQN', 'ALQ101', 'ALQ110', 'ALQ130', 'SMQ020', 'RIAGENDR', 'RIDAGEYR',
       'RIDRETH1', 'DMDCITZN', 'DMDEDUC2', 'DMDMARTL', 'DMDHHSIZ', 'WTINT2YR',
       'SDMVPSU', 'SDMVSTRA', 'INDFMPIR', 'BPXSY1', 'BPXDI1', 'BPXSY2',
       'BPXDI2', 'BMXWT', 'BMXHT', 'BMXBMI', 'BMXLEG', 'BMXARML', 'BMXARMC',
       'BMXWAIST', 'HIQ210'],
      dtype='object')

### Investigating and Cleaning Data

Create a new column named 'SMQ020x' and store data from column 'SMQ020' with following replacements:

- 1 to "Yes"
- 2 to "No"
- 7 to NaN
- 9 to NaN

In [17]:
da['SMQ020'] 

0       1
1       1
2       1
3       2
4       2
       ..
5730    1
5731    2
5732    1
5733    1
5734    2
Name: SMQ020, Length: 5735, dtype: int64

In [7]:
da['SMQ020x'] = (da['SMQ020'].replace([1, 2, 7, 9],['Yes', 'No', np.nan, np.nan]))
da['SMQ020x']

0       Yes
1       Yes
2       Yes
3        No
4        No
       ... 
5730    Yes
5731     No
5732    Yes
5733    Yes
5734     No
Name: SMQ020x, Length: 5735, dtype: object

Create a new column named 'RIAGENDRx' and store data from column 'RIAGENDR' with following replacements:

- 1 to "Male"
- 2 to "Female"

In [19]:
da['RIAGENDR']

0       1
1       1
2       1
3       2
4       2
       ..
5730    2
5731    1
5732    2
5733    1
5734    2
Name: RIAGENDR, Length: 5735, dtype: int64

In [5]:
da['RIAGENDRx'] = (da['RIAGENDR'].replace([1, 2],['Male', 'Female']))
da['RIAGENDRx']

0         Male
1         Male
2         Male
3       Female
4       Female
         ...  
5730    Female
5731      Male
5732    Female
5733      Male
5734    Female
Name: RIAGENDRx, Length: 5735, dtype: object

Drop all NAs from both `SMQ020x` & `RIAGENDRx` and store into a new dataframe named 'dx'. Plot the following crosstab using `pd.crosstab` library.

In [23]:
dx = (da[['SMQ020x','RIAGENDRx']].copy())
dx = dx.dropna(subset=['SMQ020x','RIAGENDRx'])
dx

,SMQ020x,RIAGENDRx
0,Yes,Male
1,Yes,Male
2,Yes,Male
3,No,Female
4,No,Female
...,...,...
5730,Yes,Female
5731,No,Male
5732,Yes,Female
5733,Yes,Male


In [9]:
dx.isnull

<bound method DataFrame.isnull of      SMQ020x RIAGENDRx
0        Yes      Male
1        Yes      Male
2        Yes      Male
3         No    Female
4         No    Female
...      ...       ...
5730     Yes    Female
5731      No      Male
5732     Yes    Female
5733     Yes      Male
5734      No    Female

[5725 rows x 2 columns]>

In [20]:
dx = pd.crosstab(index=dx['SMQ020x'], columns=dx['RIAGENDRx'])
dx

RIAGENDRx,Female,Male
SMQ020x,,
No,2066,1340
Yes,906,1413


Replace `dx['SMQ020x']` "Yes" to 1 and "No" to 0.

In [25]:
dx['SMQ020x'] = (dx['SMQ020x'].replace(['Yes', 'No'],['1', '0'])).astype(int)
dx

,SMQ020x,RIAGENDRx
0,1,Male
1,1,Male
2,1,Male
3,0,Female
4,0,Female
...,...,...
5730,1,Female
5731,0,Male
5732,1,Female
5733,1,Male


In [293]:
dx.dtypes

SMQ020x       int32
RIAGENDRx    object
dtype: object

Calculate the 'mean' and 'size' and store into a new dataframe called `dz`

In [39]:
mean = (dx.groupby(['RIAGENDRx'])['SMQ020x'].mean())
mean

RIAGENDRx
Female    0.304845
Male      0.513258
Name: SMQ020x, dtype: float64

In [40]:
size = (dx.groupby(['RIAGENDRx'])['SMQ020x'].size())
size

RIAGENDRx
Female    2972
Male      2753
Name: SMQ020x, dtype: int64

In [41]:
dz = pd.concat([mean, size],axis=1)
dz.columns = ['mean', 'size']
dz

,mean,size
RIAGENDRx,,
Female,0.304845,2972
Male,0.513258,2753


In [42]:
# from sir

dz = dx.groupby('RIAGENDRx').agg({'SMQ020x':[lambda x:np.mean(x==1),np.size]})
dz.columns = ['mean', 'size']
dz

,mean,size
RIAGENDRx,,
Female,0.304845,2972
Male,0.513258,2753


### Constructing Confidence Intervals

Now that we have the population proportions of male and female smokers, we can begin to calculate confidence intervals.  From lecture, we know that the equation is as follows:

$$Best\ Estimate \pm Margin\ of\ Error$$

Where the *Best Estimate* is the **observed population proportion or mean** from the sample and the *Margin of Error* is the **t-multiplier**.

The equation to create a 95% confidence interval can also be shown as:

$$Population\ Proportion\ or\ Mean\ \pm (t-multiplier *\ Standard\ Error)$$

The Standard Error is calculated differenly for population proportion and mean:

$$Standard\ Error \ for\ Population\ Proportion = \sqrt{\frac{Population\ Proportion * (1 - Population\ Proportion)}{Number\ Of\ Observations}}$$

$$Standard\ Error \ for\ Mean = \frac{Standard\ Deviation}{\sqrt{Number\ Of\ Observations}}$$

Lastly, the standard error for difference of population proportions and means is:

$$Standard\ Error\ for\ Difference\ of\ Two\ Population\ Proportions\ Or\ Means = \sqrt{SE_{Proportion\ 1}^2 + SE_{Proportion\ 2} ^2}$$

#### Difference of Two Population Proportions

Calculate the standard error for female

In [43]:
#0.00844
mf = dz['mean']['Female']
sf = dz['size']['Female']
sef = np.sqrt(mf*(1-mf)/sf)
sef

0.008444152146214435

Calculate the standard error for male

In [45]:
#0.00952
mm = dz['mean']['Male']
sm = dz['size']['Male']
sem = np.sqrt(mm*(1-mm)/sm)
sem

0.009526078653689868

Calculate the difference between these two Standard Errors

In [46]:
#0.0127988
dstde = np.sqrt(sem**2 + sef**2)
dstde

0.012729881381407434

Calculate the confidence Interval

In [48]:
# Slide 42 Confidence Interval

In [47]:
pci = dz['mean']['Female'] - dz['mean']['Male']
x = pci - 2*dstde
y = pci + 2*dstde

print(x,y)

-0.2338728044024504 -0.18295327887682067


#### Difference of Two Population Means

Now we look into the differences between 2 population means

In [34]:
da["BMXBMI"].head()

0    27.8
1    30.8
2    28.8
3    42.4
4    20.3
Name: BMXBMI, dtype: float64

In [50]:
populations = da.groupby(['RIAGENDRx']).agg({'BMXBMI':[np.mean, np.std, np.size]})
populations

BMXBMI                  
                mean       std    size
RIAGENDRx                             
Female     29.939946  7.753319  2976.0
Male       28.778072  6.252568  2759.0

Calculate the Standard Error for Mean for both female and male

In [54]:
f = 7.753/np.sqrt(dz['size']['Female'])
m = 6.253/np.sqrt(dz['size']['Male'])
print (f,m)

0.14221499207323504 0.11917504456014921


Calculate the difference between 2 Standard Error for Mean

In [55]:
e = np.sqrt(f**2 + m**2)
e

0.18554728566137488

The difference between two means for male and female

In [57]:
mean = populations[('BMXBMI','mean')]['Female'] - populations[('BMXBMI','mean')]['Male']
mean

1.1618735403270115

Calculate the confidence interval between two population means

In [56]:
a = mean - 2*e
b = mean + 2*e
print(a,b)

58.346923192698235 59.089112335343735
